In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import time
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, precision_recall_curve, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from imblearn.over_sampling import SMOTE

In [2]:
df_test = pd.read_csv('/Users/mikeredshaw/Documents/Schulich MBAN/Data Science II/Assignment 1/test.csv', ";")
df_train = pd.read_csv('/Users/mikeredshaw/Documents/Schulich MBAN/Data Science II/Assignment 1/train.csv', ";")

/var/folders/4r/t2pxznrn4dz28d6ymprq1p9c0000gn/T/ipykernel_16656/2530379859.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df_test = pd.read_csv('/Users/mikeredshaw/Documents/Schulich MBAN/Data Science II/Assignment 1/test.csv', ";")
/var/folders/4r/t2pxznrn4dz28d6ymprq1p9c0000gn/T/ipykernel_16656/2530379859.py:2: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df_train = pd.read_csv('/Users/mikeredshaw/Documents/Schulich MBAN/Data Science II/Assignment 1/train.csv', ";")


In [3]:
df_test.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [4]:
df_train.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [6]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        4521 non-null   int64 
 1   job        4521 non-null   object
 2   marital    4521 non-null   object
 3   education  4521 non-null   object
 4   default    4521 non-null   object
 5   balance    4521 non-null   int64 
 6   housing    4521 non-null   object
 7   loan       4521 non-null   object
 8   contact    4521 non-null   object
 9   day        4521 non-null   int64 
 10  month      4521 non-null   object
 11  duration   4521 non-null   int64 
 12  campaign   4521 non-null   int64 
 13  pdays      4521 non-null   int64 
 14  previous   4521 non-null   int64 
 15  poutcome   4521 non-null   object
 16  y          4521 non-null   object
dtypes: int64(7), object(10)
memory usage: 600.6+ KB


In [7]:
df_test['y'] = df_test['y'].map({'yes': 1, 'no': 0})
df_train['y'] = df_train['y'].map({'yes': 1, 'no': 0})

In [8]:
unknown_counts_train = (df_train == 'unknown').sum()
columns_with_unknown_train = unknown_counts_train[unknown_counts_train > 0]
proportion_with_unknown_train = unknown_counts_train[unknown_counts_train > 0] / len(df_train)
print(columns_with_unknown_train,  proportion_with_unknown_train*100)

job            288
education     1857
contact      13020
poutcome     36959
dtype: int64 job           0.637013
education     4.107407
contact      28.798301
poutcome     81.747805
dtype: float64


In [9]:
unknown_counts_test = (df_test == 'unknown').sum()
columns_with_unknown_test = unknown_counts_test[unknown_counts_test > 0]
proportion_with_unknown_test = unknown_counts_test[unknown_counts_test > 0] / len(df_test)
print(columns_with_unknown_test,  proportion_with_unknown_test*100)

job            38
education     187
contact      1324
poutcome     3705
dtype: int64 job           0.840522
education     4.136253
contact      29.285556
poutcome     81.950896
dtype: float64


In [10]:
df_train['job'].replace('unknown', df_train['job'].mode()[0], inplace=True)
df_train['education'].replace('unknown', df_train['education'].mode()[0], inplace=True)
df_test['job'].replace('unknown', df_test['job'].mode()[0], inplace=True)
df_test['education'].replace('unknown', df_test['education'].mode()[0], inplace=True)

In [11]:
contact_train = df_train[df_train['contact'] != 'unknown'] # setting the training data to be the rows that do not have unknown in the contact column
contact_train = contact_train.drop('y', axis=1) #removing y from the data as this would not be available for imputation predictions
contact_predict = df_train[df_train['contact'] == 'unknown'] #setting the predict data to include the rows with unknown in the contact column
contact_predict = contact_predict.drop('y', axis=1) # removing y column once again.

In [12]:
X = contact_train.drop(['contact'], axis=1)
y= contact_train['contact']

X_train_actual, X_valid, y_train_actual, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

encoder = OneHotEncoder(handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train_actual)

X_valid_encoded = encoder.transform(X_valid)
X_predict_encoded = encoder.transform(contact_predict.drop('contact', axis=1))

In [13]:
dt = DecisionTreeClassifier()
dt.fit(X_train_encoded, y_train_actual)

y_valid_pred = dt.predict(X_valid_encoded)

accuracy = accuracy_score(y_valid, y_valid_pred)
print(f"Accuracy of imputation on the validation set: {accuracy:.2f}")

predicted_values = dt.predict(X_predict_encoded)

df_train.loc[df_train['contact'] == 'unknown', 'contact'] = predicted_values

Accuracy of imputation on the validation set: 0.88


In [14]:
contact_train_2 = df_test[df_test['contact'] != 'unknown']
contact_predict_2 = df_test[df_test['contact'] == 'unknown']

X_train_actual_2, X_valid_2, y_train_actual_2, y_valid_2 = train_test_split(
    contact_train_2.drop('contact', axis=1),
    contact_train_2['contact'],
    test_size=0.3,
    random_state=42
)

encoder = OneHotEncoder(handle_unknown='ignore')
X_train_encoded_2 = encoder.fit_transform(X_train_actual_2)

X_valid_encoded_2 = encoder.transform(X_valid_2)
X_predict_encoded_2 = encoder.transform(contact_predict_2.drop('contact', axis=1))

In [15]:
dt = DecisionTreeClassifier()
dt.fit(X_train_encoded_2, y_train_actual_2)

y_valid_pred_2 = dt.predict(X_valid_encoded_2)

accuracy = accuracy_score(y_valid_2, y_valid_pred_2)
print(f"Accuracy of imputation on the validation set: {accuracy:.2f}")

predicted_values_2 = dt.predict(X_predict_encoded_2)

df_test.loc[df_test['contact'] == 'unknown', 'contact'] = predicted_values_2

Accuracy of imputation on the validation set: 0.88


In [16]:
print('df_train contact value counts after imputation:\n', df_train['contact'].value_counts())
print('df_train contact value counts after imputation:\n', df_test['contact'].value_counts())

df_train contact value counts after imputation:
 cellular     41515
telephone     3696
Name: contact, dtype: int64
df_train contact value counts after imputation:
 cellular     4136
telephone     385
Name: contact, dtype: int64


In [17]:
df_train = df_train[df_train['previous'] <= 250]

In [18]:
cat_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'day', 'month', 'campaign','poutcome']
num_columns = ['balance', 'pdays', 'previous']
target = 'y'

In [19]:
num_grouped = df_train[num_columns].groupby(df_train['y']).mean()

def mode_agg(x):
    return x.mode().iloc[0]

cat_grouped = df_train[cat_columns].groupby(df_train['y']).agg(mode_agg)
df_train_grouped = pd.concat([num_grouped, cat_grouped], axis=1)

In [20]:
df_train_grouped.head()

,balance,pdays,previous,job,marital,education,default,housing,loan,contact,day,month,campaign,poutcome
y,,,,,,,,,,,,,,
0,1303.734025,36.415721,0.495278,blue-collar,married,secondary,no,yes,no,cellular,20,may,1,unknown
1,1804.267915,68.702968,1.170354,management,married,secondary,no,no,no,cellular,30,may,1,unknown


In [21]:
df_train.insert(14, 'contacted', np.where(df_train['pdays'] == -1, 0, 1))
df_test.insert(14, 'contacted', np.where(df_test['pdays'] == -1, 0, 1))

In [22]:
df_train.rename(columns={'loan': 'personal_loan'}, inplace=True)
df_test.rename(columns={'loan': 'personal_loan'}, inplace=True)
df_train.rename(columns={'housing': 'housing_loan'}, inplace=True)
df_test.rename(columns={'housing': 'housing_loan'}, inplace=True)

In [23]:
df_train.insert(8, 'any_loans', np.where((df_train['housing_loan'] == 'yes') | (df_train['personal_loan'] == 'yes'), 1, 0))
df_test.insert(8, 'any_loans', np.where((df_test['housing_loan'] == 'yes') | (df_test['personal_loan'] == 'yes'), 1, 0))

In [24]:
df_train.insert(9, 'both_loans', np.where((df_train['housing_loan'] == 'yes') & (df_train['personal_loan'] == 'yes'), 1, 0))
df_test.insert(9, 'both_loans', np.where((df_test['housing_loan'] == 'yes') & (df_test['personal_loan'] == 'yes'), 1, 0))

In [27]:
df_train.describe(include='all')

,age,job,marital,education,default,balance,housing_loan,personal_loan,any_loans,both_loans,contact,day,month,duration,campaign,pdays,contacted,previous,poutcome,y
count,45210.000000,45210,45210,45210,45210,45210.000000,45210,45210,45210.000000,45210.000000,45210,45210.000000,45210,45210.000000,45210.000000,45210.000000,45210.000000,45210.000000,45210,45210.000000
unique,NaN,11,3,3,2,NaN,2,2,NaN,NaN,2,NaN,12,NaN,NaN,NaN,NaN,NaN,4,NaN
top,NaN,blue-collar,married,secondary,no,NaN,yes,no,NaN,NaN,cellular,NaN,may,NaN,NaN,NaN,NaN,NaN,unknown,NaN
freq,NaN,10020,27213,25059,44395,NaN,25129,37966,NaN,NaN,41514,NaN,13766,NaN,NaN,NaN,NaN,NaN,36959,NaN
mean,40.936231,NaN,NaN,NaN,NaN,1362.290179,NaN,NaN,0.619465,0.096594,NaN,15.806724,NaN,258.161071,2.763858,40.192922,0.182614,0.574253,NaN,0.116987
std,10.618879,NaN,NaN,NaN,NaN,3044.797065,NaN,NaN,0.485524,0.295407,NaN,8.322315,NaN,257.530306,3.098053,100.124419,0.386354,1.907928,NaN,0.321409
min,18.000000,NaN,NaN,NaN,NaN,-8019.000000,NaN,NaN,0.000000,0.000000,NaN,1.000000,NaN,0.000000,1.000000,-1.000000,0.000000,0.000000,NaN,0.000000
25%,33.000000,NaN,NaN,NaN,NaN,72.000000,NaN,NaN,0.000000,0.000000,NaN,8.000000,NaN,103.000000,1.000000,-1.000000,0.000000,0.000000,NaN,0.000000
50%,39.000000,NaN,NaN,NaN,NaN,448.000000,NaN,NaN,1.000000,0.000000,NaN,16.000000,NaN,180.000000,2.000000,-1.000000,0.000000,0.000000,NaN,0.000000
75%,48.000000,NaN,NaN,NaN,NaN,1428.000000,NaN,NaN,1.000000,0.000000,NaN,21.000000,NaN,319.000000,3.000000,-1.000000,0.000000,0.000000,NaN,0.000000


In [25]:
df_train.head()

,age,job,marital,education,default,balance,housing_loan,personal_loan,any_loans,both_loans,contact,day,month,duration,campaign,pdays,contacted,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,1,0,cellular,5,may,261,1,-1,0,0,unknown,0
1,44,technician,single,secondary,no,29,yes,no,1,0,cellular,5,may,151,1,-1,0,0,unknown,0
2,33,entrepreneur,married,secondary,no,2,yes,yes,1,1,cellular,5,may,76,1,-1,0,0,unknown,0
3,47,blue-collar,married,secondary,no,1506,yes,no,1,0,cellular,5,may,92,1,-1,0,0,unknown,0
4,33,blue-collar,single,secondary,no,1,no,no,0,0,cellular,5,may,198,1,-1,0,0,unknown,0


In [26]:
def subscribe_rate(df, column):

    grouped = df.groupby([column, 'y']).size().unstack()
    grouped['percentage'] = (grouped[1] / grouped.sum(axis=1)) * 100
    result = grouped['percentage']
    print(result)

In [31]:
cat_columns = ['job', 'marital', 'education', 'default', 'housing_loan', 'personal_loan', 'contact', 'day', 'month','any_loans', 'both_loans']
num_columns = ['balance', 'pdays', 'previous', 'campaign', 'duration']
target = 'y'

In [34]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_columns)])

In [32]:
X_train = df_train[cat_columns + num_columns]
X_test = df_test[cat_columns + num_columns]
y_train = df_train[target]
y_test = df_test[target]

In [35]:
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

In [36]:
smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_preprocessed, y_train)

In [37]:
print(y_train_resampled.value_counts())

0    39921
1    39921
Name: y, dtype: int64


In [43]:
knn = KNeighborsClassifier(n_neighbors=10)
log_reg = LogisticRegression()
dt = DecisionTreeClassifier(max_depth=20)
rf = RandomForestClassifier()
ada = AdaBoostClassifier()
bag = BaggingClassifier()
voting = VotingClassifier(estimators=[('lr', log_reg), ('knn', knn), ('dt', dt)])

In [44]:
classifiers = {
    'K-Nearest Neighbors': knn,
    'Logistic Regression': log_reg,
    'Decision Tree': dt,
    'Random Forest': rf,
    'AdaBoost': ada,
    'Bagging': bag,
    'Voting': voting
}

In [41]:
results = {}

In [46]:
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

In [47]:
# Import additional libraries
from sklearn.model_selection import GridSearchCV

# Hyperparameter grids for tuning
knn_params = {'classifier__n_neighbors': [3, 5, 7, 20, 30, 50, 100]}
log_reg_params = {'classifier__C': [0.1, 1, 10]}
dt_params = {'classifier__max_depth': [10,20,30,40,50]}
rf_params = {'classifier__n_estimators': [50, 100, 150], 'classifier__max_depth': [None, 10, 20, 30, 50]}
ada_params = {'classifier__n_estimators': [25, 50, 75]}
bag_params = {'classifier__n_estimators': [5, 10, 20]}
voting_params = {'classifier__voting': ['hard', 'soft']}

params_dict = {
    'K-Nearest Neighbors': knn_params,
    'Logistic Regression': log_reg_params,
    'Decision Tree': dt_params,
    'Random Forest': rf_params,
    'AdaBoost': ada_params,
    'Bagging': bag_params,
    'Voting': voting_params
}

# Initialize results dictionary for tuned models
tuned_results = {}

# Loop through classifiers for tuning
for name, clf in classifiers.items():
    start_time = time.time()
    
    # Create pipeline
    pipeline = Pipeline(steps=[('classifier', clf)])
    
    # Create GridSearchCV object
    grid = GridSearchCV(pipeline, params_dict[name], cv=5)
    
    # Fit the model
    grid.fit(X_train_resampled, y_train_resampled)
    
    # Get the best estimator and predict
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test_preprocessed)
    
    # Compute metrics
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    # Store results
    tuned_results[name] = {
        'Best Params': grid.best_params_,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'Time (s)': elapsed_time
    }

# Convert results to DataFrame for easier viewing
tuned_results_df = pd.DataFrame(tuned_results).T
print(tuned_results_df);

KeyboardInterrupt: 